In [1]:
import os
import sys
p = os.path.abspath('../..')
if p not in sys.path:
    sys.path.append(p)

from waveorder.io.writer import WaveorderWriter
import numpy as np
import zarr

In [2]:
writer = WaveorderWriter('/Users/cameron.foltz/Desktop/zarr_test', 'physical')

In [3]:
writer.set_position(2)
writer.create_zarr()

Found existing store at /Users/cameron.foltz/Desktop/zarr_test/Pos_002/physical_data.zarr


In [4]:
T = 1
C = 2
Z = 65
Y = 2048
X = 2048

data = np.ones((T, C, Z, Y, X))

In [5]:
writer.init_array(data_shape=data.shape, chunk_size=(1,1,1,2048,2048), dtype='uint16')

In [4]:
writer.set_channel_attributes(['DAPI', 'Phase3D'])

In [5]:
writer.store.attrs.asdict()

{'multiscales': [{'datasets': [{'path': 'array'}], 'version': '0.1'}],
 'omero': {'channels': [{'active': True,
    'coefficient': 1.0,
    'color': '808080',
    'family': 'linear',
    'inverted': False,
    'label': 'DAPI',
    'window': {'end': 1.0, 'max': 65535.0, 'min': 0.0, 'start': 0.0}},
   {'active': True,
    'coefficient': 1.0,
    'color': '808080',
    'family': 'linear',
    'inverted': False,
    'label': 'Phase3D',
    'window': {'end': 1.0, 'max': 65535.0, 'min': 0.0, 'start': 0.0}}]}}

In [11]:
writer.write(data[0, 0, 0:65, :, :], T=[0], C=[0], Z=[0,65])

In [16]:
np.shape(data[0:2, 0:50, 0:65, :, :])

(2, 1, 65, 2048, 2048)

In [ ]:
writer.add_store('func_test_2')

In [ ]:
writer.add_store('func_test_3.zarr')

In [ ]:
writer.set_store('func_test_2')

In [18]:
print(writer.store.info)

Name        : /
Type        : zarr.hierarchy.Group
Read-only   : False
Store type  : zarr.storage.DirectoryStore
No. members : 1
No. arrays  : 1
No. groups  : 0
Arrays      : array



In [9]:
example = zarr.open('/Users/cameron.foltz/6001240.zarr')

In [10]:
example.attrs.asdict()

{'_creator': {'name': 'omero-zarr',
  'version': '0.0.10.dev0+g98cf12c.d20201130'},
 'multiscales': [{'datasets': [{'path': '0'}, {'path': '1'}, {'path': '2'}],
   'version': '0.1'}],
 'omero': {'channels': [{'active': True,
    'coefficient': 1.0,
    'color': '0000FF',
    'family': 'linear',
    'inverted': False,
    'label': 'LaminB1',
    'window': {'end': 1500.0, 'max': 65535.0, 'min': 0.0, 'start': 0.0}},
   {'active': True,
    'coefficient': 1.0,
    'color': 'FFFF00',
    'family': 'linear',
    'inverted': False,
    'label': 'Dapi',
    'window': {'end': 1500.0, 'max': 65535.0, 'min': 0.0, 'start': 0.0}}],
  'id': 1,
  'rdefs': {'defaultT': 0, 'defaultZ': 118, 'model': 'color'},
  'version': '0.1'}}